# Tutorial: Using `fastai.data` low-level APIs

> Using `DataSource`, `Pipeline`, `TfmdList`, `TfmOver`, and `Transform`

In [ ]:
from local.imports import *
from local.test import *
from local.core import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.vision.core import *
from local.data.external import *

from PIL.ImageFile import ImageFile

In [ ]:
#hide
torch.cuda.set_device(int(os.environ.get('DEFAULT_GPU') or 0))

## setup

In [ ]:
def mk_tensor(im): return tensor(array(im))[None]

@patch
def dihedral(im:ImageFile, idx=0): return im.transpose(idx-1) if idx>=1 else im
def rand_dihedral(im): return im.dihedral(random.randint(0,7))

def normalize  (o,m,s): return (o-m)/s
def denormalize(o,m,s): return (o*s)+m

def decode_vocab(o, v): return [v[o_] for o_ in o]

In [ ]:
source = untar_data(URLs.MNIST_TINY)/'train'
items = get_image_files(source)
fn = items[0]
img = PIL.Image.open(fn)
m,s = 0.1,0.3

In [ ]:
imgt = mk_tensor(img)

lbls = items.mapped(parent_label)
itos,stoi = uniqueify(lbls, bidir=True, sort=True)

## v6

In [ ]:
class Transform(PrePostInit):
    def __init__(self,encode=None,decode=None):
        if encode: self.encode = encode
        if decode: self.decode = decode

In [ ]:
@patch
def powx(x:math, a): return math.pow(x,a)

@patch
def powx(x:torch, a): return torch.pow(x,a)

class Add1(Transform):
    def encode(self,x): return x+1
    def decode(self,x): return x-1

def mk_torch(x)->torch: return tensor(x)
def mk_int  (x)->math : return x

In [ ]:
class Func():
    def __init__(self, nm, *args, **kwargs): self.nm,self.args,self.kwargs = nm,args,kwargs
    def __repr__(self): return f'sig: {self.nm}({self.args}, {self.kwargs})'
    def __call__(self,t):
        f = getattr(t,self.nm)
        if not (self.args or self.kwargs): return f
        return partial(f, *self.args, **self.kwargs)
    
class SelfFunc():
    def __init__(self, nm, *args, **kwargs): self.nm,self.args,self.kwargs = nm,args,kwargs
    def __repr__(self): return f'self: {self.nm}({self.args}, {self.kwargs})'
    def __call__(self, o):
        return getattr(o,self.nm)(*self.args, **self.kwargs)
    
class _Sig():
    def __getattr__(self,k):
        def _inner(*args, **kwargs):
            return Func(k, *args, **kwargs)
        return _inner

class _SelfFunc():
    def __getattr__(self,k):
        def _inner(*args, **kwargs):
            return SelfFunc(k, *args, **kwargs)
        return _inner

Sig = _Sig()
Self = _SelfFunc()

In [ ]:
pipe_funcs = [Add1(), (Sig.powx(a=2), 'sqrt'), mk_torch, (Self.reciprocal(),Self.reciprocal())]

pf1 = [mk_int  ] + pipe_funcs
pf2 = [mk_torch] + pipe_funcs

In [ ]:
def mk_func(f, t):
    if isinstance(f,str ): f = Func(f)
    if isinstance(f,Func): f = f(t)
    return f

def mk_tfm(f,t):
    if not is_listy(f): f = (f,None)
    return Transform(mk_func(f[0],t), mk_func(f[1],t))

In [ ]:
def compose_tfms(x, tfms, func_nm='encode', reverse=False):
    if reverse: tfms = reversed(tfms)
    for tfm in tfms: x = getattr(tfm,func_nm,noop)(x)
    return x

In [ ]:
def _get_ret(func):
    ann = getattr(func,'__annotations__', None)
    if not ann: return None
    return ann.get('return')

In [ ]:
class Pipeline():
    def __init__(self, funcs):
        self.fs = []
        self.t = None
        for f in funcs:
            if not isinstance(f,Transform): f = mk_tfm(f, self.t)
            self.fs.append(f)
            self.t = _get_ret(f.encode) or self.t
                
    def __call__(self, o): return compose_tfms(o, self.fs)
    def decode  (self, i): return compose_tfms(i, self.fs, func_nm='decode', reverse=True)

In [ ]:
p1 = Pipeline(pf1)
a1 = p1(1.5); a1

tensor(0.1600)

In [ ]:
p1.decode(a1)

1.5

In [ ]:
p2 = Pipeline(pf2)
a2 = p2(tensor(1.5)); a2

tensor(0.1600)

In [ ]:
p2.decode(a2)

tensor(1.5000)

## fin